In [1]:
import gzip
from collections import defaultdict
import numpy as np
import random
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
def readGz(f):
  for l in gzip.open(f):
    yield eval(l)
    
print "Reading data..."
data = list(readGz('train.json.gz'))
print "Done"

Reading data...
Done


In [30]:
data[7777]

{'categories': [['Clothing, Shoes & Jewelry',
   'Women',
   'Clothing',
   'Dresses',
   'Casual'],
  ['Clothing, Shoes & Jewelry', 'Women', 'Plus-Size']],
 'categoryID': 0,
 'helpful': {'nHelpful': 1, 'outOf': 1},
 'itemID': 'I006212764',
 'rating': 3.0,
 'reviewHash': 'R595520643',
 'reviewText': '...but if someone were to wear it as a dress, they would freeze their business.  And they would deserve to.It does make a nice warm sweater with jeans or a skirt, though.',
 'reviewTime': '12 27, 2012',
 'reviewerID': 'U651319519',
 'summary': 'Makes for a nice sweater...',
 'unixReviewTime': 1356566400}

In [33]:
data[777]

{'categories': [['Clothing, Shoes & Jewelry',
   'Women',
   'Clothing',
   'Lingerie, Sleep & Lounge',
   'Intimates',
   'Shapewear',
   'Waist Cinchers'],
  ['Clothing, Shoes & Jewelry',
   'Women',
   'Petite',
   'Intimates',
   'Shapewear',
   'Waist Cinchers'],
  ['Clothing, Shoes & Jewelry',
   'Women',
   'Plus-Size',
   'Intimates',
   'Shapewear',
   'Waist Cinchers']],
 'categoryID': 0,
 'helpful': {'nHelpful': 0, 'outOf': 0},
 'itemID': 'I490564816',
 'rating': 4.0,
 'reviewHash': 'R072101032',
 'reviewText': "This corset in addition to a few more helped me get back my pre pregnancy body after having twins. I would put this one over a compression garment and continued to go down in sizes until I reached small. Eventually I no longer needed a corset (I'm size 1 now). Highly recommend, however if you have a lot of fat on your back it will spill over",
 'reviewTime': '03 5, 2014',
 'reviewerID': 'U667520612',
 'summary': 'Great product',
 'unixReviewTime': 1393977600}

In [ ]:
# the basic idea is that we judge whether a item will be purchased or not
# by the similarity of items bought by user and items didn't bought by user

In [59]:
# we first get what items user has bought
user_item = defaultdict(set)
for d in data:
    u = d['reviewerID']
    user_item[u].add(d['itemID'])

In [4]:
# caculate the mean number of items user bought
# to get a sense of how many items i should generate for
# non_purchase items for each user


'''
mean_num = 0
user_num = 0
for u in user_item:
    user_num += 1
    mean_num += len(user_item[u])
mean_num /= user_num
mean_num
'''

5

In [62]:
# get all item info
item_info = defaultdict(list)
for d in data:
    i = d['itemID']
    if not i in item_info:
        for cate in d['categories']:
            item_info[i].append(cate)

In [63]:
len(item_info)

19914

In [107]:
# generate the non_purchased item for each user
non_user_item = defaultdict(set)
for u in user_item:
    while len(non_user_item[u]) < len(user_item[u]):
        i = random.choice(item_info.keys())
        if i not in user_item[u]:
            non_user_item[u].add(i)

In [65]:
# get the similarity
def get_similarity(u, i, u_i):
    if i in u_i[u]:
        return 1
    # just use Jaccard similarity: J(A,B) = |A and B| / |A or B|
    all_cate = list()
    for item in u_i[u]:
        for cate_list in item_info[item]:
            for c in cate_list:
                if c not in all_cate:
                    all_cate.append(c)
    i_cate = list()
    for cate_list in item_info[i]:
        for c in cate_list:
            i_cate.append(c)
    a_and_b = 0
    a_or_b = 0
    for c in i_cate:
        if c in all_cate:
            a_and_b += 1
        else:
            a_or_b += 1
    a_or_b += len(all_cate)
    return a_and_b / (a_or_b * 1.0)
    

In [66]:
# judge whether user will buy this item
def will_purchase(u,i):
    if (u not in user_item) or (i not in item_info):
        return random.randint(0,1)
    y = get_similarity(u,i,user_item)
    n = get_similarity(u,i,non_user_item)
    if y > n:
        return 1
    else:
        return 0

In [108]:
# get the prediction
predictions = open("/Users/nan/Desktop/assign1_pairs_Purchase.txt", 'w')
for l in open("pairs_Purchase.txt"):
    if l.startswith("reviewerID"):
        #header
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    predictions.write(u + '-' + i + ',' + str(will_purchase(u,i)) + '\n')
predictions.close()

In [61]:
len(item_info)

19915

In [68]:
len(user_item)

39239

In [72]:
a = dict()
if '1' not in a:
    a['1'] = []
a['1'].append(1)
a['1'].append(4)

In [73]:
a

{'1': [1, 4]}

In [76]:
a = set()
a.add(1)
a.add(2)
a.add(3)
for i in a:
    print i
for i in a:
    print i

1
2
3
1
2
3


In [91]:
def getKey(i):
    return i[1]
a = [('ssqqd',11),('ads',2)]
sorted(a,key=getKey)
a

[[2, 3], [6, 7], [3, 34], [24, 64], [1, 43]]

In [92]:
a = [1,2,4,3,7]
sorted(a)

[1, 2, 3, 4, 7]

In [93]:
a = [[2, 3], [6, 7], [3, 34], [24, 64], [1, 43]]
sorted(a,key=getKey)

[[2, 3], [6, 7], [3, 34], [1, 43], [24, 64]]

In [94]:
sorted([('abc', 121),('abc', 231),('abc', 148), ('abc',221)], key=lambda x: x[1])

[('abc', 121), ('abc', 148), ('abc', 221), ('abc', 231)]

In [101]:
a = sorted([('ssqqd',11),('ads',2)], key=lambda x: x[0],reverse=True)

In [103]:
a = random.choice(data)

In [104]:
a

{'categories': [['Clothing, Shoes & Jewelry',
   'Women',
   'Clothing',
   'Socks & Hosiery',
   'Dress & Trouser Socks']],
 'categoryID': 0,
 'helpful': {'nHelpful': 0, 'outOf': 0},
 'itemID': 'I789706599',
 'price': 18.0,
 'rating': 4.0,
 'reviewHash': 'R682851280',
 'reviewText': "These fit fine, but slid down my legs after probably 30 steps. I know I have big calves, but the elastic in these just isn't that great. But, with some sock glue they stayed up just fine.",
 'reviewTime': '06 17, 2013',
 'reviewerID': 'U991841502',
 'summary': 'You will definitely need some sock glue',
 'unixReviewTime': 1371427200}

In [105]:
a = [1,2,3]
a = a[1:]

In [106]:
a

[2, 3]